# Acquisition

Acquisition automatisée de 60 images de chats et 60 images de chiens depuis Wikidata dans le dossier "/images"

In [3]:
import sys
from SPARQLWrapper import SPARQLWrapper, JSON
import pandas as pd

endpoint_url = "https://query.wikidata.org/sparql"

In [4]:
query = """SELECT DISTINCT ?animal ?animalLabel ?paysdOrigine ?paysdOrigineLabel ?image WHERE {
  { 
    SELECT ?animal ?paysdOrigine (SAMPLE(?img) AS ?image) WHERE {
      ?animal wdt:P31 wd:Q43577;  # Chats
              wdt:P495 ?paysdOrigine;
              wdt:P18 ?img.
    } GROUP BY ?animal ?paysdOrigine
    LIMIT 60
  }
  UNION
  { 
    SELECT ?animal ?paysdOrigine (SAMPLE(?img) AS ?image) WHERE {
      ?animal wdt:P31 wd:Q39367;  # Chiens
              wdt:P495 ?paysdOrigine;
              wdt:P18 ?img.
    } GROUP BY ?animal ?paysdOrigine
    LIMIT 60
  }

  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}"""


def get_results(endpoint_url, query):
    user_agent = "WDQS-example Python/%s.%s" % (
        sys.version_info[0],
        sys.version_info[1],
    )
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()

array = []
results = get_results(endpoint_url, query)

for result in results["results"]["bindings"]:
    array.append(
        (
            result["animalLabel"]["value"],
            result["paysdOrigineLabel"]["value"],
            result["image"]["value"],
        )
    )

In [5]:
dataframe = pd.DataFrame(array, columns=["animal", "pays d'origine", "image"])
dataframe = dataframe.astype(
    dtype={"animal": "<U200", "pays d'origine": "<U200", "image": "<U200"}
)
dataframe

,animal,pays d'origine,image
0,Cymric,Canada,http://commons.wikimedia.org/wiki/Special:File...
1,LaPerm,United States,http://commons.wikimedia.org/wiki/Special:File...
2,Abyssinian,United Kingdom,http://commons.wikimedia.org/wiki/Special:File...
3,Kinkalow,United States,http://commons.wikimedia.org/wiki/Special:File...
4,Oriental Shorthair,United States,http://commons.wikimedia.org/wiki/Special:File...
...,...,...,...
115,Alaskan Klee Kai,United States,http://commons.wikimedia.org/wiki/Special:File...
116,Picardy Spaniel,France,http://commons.wikimedia.org/wiki/Special:File...
117,Bergamasco Shepherd,Italy,http://commons.wikimedia.org/wiki/Special:File...
118,Löwchen,Europe,http://commons.wikimedia.org/wiki/Special:File...


In [6]:
import requests
import shutil
import os

def download_image(url):
    headers = {"User-Agent": "Mozilla/5.0"}
    request = requests.get(url, allow_redirects=True, headers=headers, stream=True)
    if request.status_code == 200:
        with open("images/"+os.path.basename(url), "wb") as image:
            request.raw.decode_content = True
            shutil.copyfileobj(request.raw, image)
    return request.status_code

In [7]:
dataframe.image.apply(download_image)

0      200
1      200
2      200
3      200
4      200
      ... 
115    200
116    200
117    200
118    200
119    200
Name: image, Length: 120, dtype: int64

On renomme les noms des images pour faciler leur traitement

In [8]:
import os

dossier_images = "images/"  

# Lister tous les fichiers dans le dossier
images = [f for f in os.listdir(dossier_images) if os.path.isfile(os.path.join(dossier_images, f))]

# Renommer les images
for i, image in enumerate(images, 1):  
    # Obtenir l'extension de l'image et créer un nouveau nom pour l'image
    _, extension = os.path.splitext(image)
    nouveau_nom = f"{i}{extension}"
    
    # Chemin complet de l'image avant et après renaming
    chemin_image_avant = os.path.join(dossier_images, image)
    chemin_image_apres = os.path.join(dossier_images, nouveau_nom)
    
    # Renommer l'image
    os.rename(chemin_image_avant, chemin_image_apres)
    
    print(f"Image {image} renommée en {nouveau_nom}")


Image %282%29%20Isha%20female%20rajapalayam.jpg renommée en 1.jpg
Image %D0%A4%D0%BE%D1%82%D0%BE%20%D0%BA%D1%83%D0%BD%D0%BE%D0%B2.jpg renommée en 2.jpg
Image 09057110%20Jura%20Laufhund.jpg renommée en 3.jpg
Image 618736210a13.jpg renommée en 4.jpg
Image 8-month-old%20male%20Lykoi.jpg renommée en 5.jpg
Image Abessinierkater1.jpg renommée en 6.jpg
Image Adult%20Glen%20of%20Imaal%20terrier.jpg renommée en 7.jpg
Image Aegean%20Island%20Cat.jpg renommée en 8.jpg
Image Akbash%20Dog%20in%20CA.jpg renommée en 9.jpg
Image American%20bobtail%202.jpg renommée en 10.jpg
Image American%20curl%202.jpg renommée en 11.jpg
Image American%20Wirehair%20-%20CFF%20cat%20show%20Heinola%202008-05-04%20IMG%208721.JPG renommée en 12.JPG
Image AmericanCockerSpaniel%20wb.jpg renommée en 13.jpg
Image Armenian%20Gampr%2001.jpg renommée en 14.jpg
Image Australian%20Mist.jpg renommée en 15.jpg
Image BebopsLilacPrince.JPG renommée en 16.JPG
Image Bergamasco%20600.jpg renommée en 17.jpg
Image Betty%20Verdure.Photo%20P